In [4]:
%load_ext autoreload
%autoreload 2

import jCMIP as jc
import glob

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
cmip = '6'
EXP  = 'piControl'
lag  = '-3,-2,-1,0,1,2,3,4,5'

var  = 'zos'
season_field = ['DJF', 'MAM', 'JJA', 'SON']

var_ts       = ['max_vals_eastward_wind_', 'max_lats_eastward_wind_']
nvar_ts      = ['max_wind', 'max_lats']
season_ts    = ['DJF', 'MAM', 'JJA', 'SON']
BenType = '0to60W_fromseasmean'

# Directory/file details:
dataBen = '/gws/nopw/j04/acsis/bjharvey/storylines/data/JetDiags/'
BenType = '0to60W_fromseasmean'
datadir = '/gws/nopw/j04/acsis/jmecking/CMIP6/'

In [16]:
for sf in season_field:
    for st in season_ts:
        print(sf + '  ' + st)
        
        # Output file:
        outfile = (datadir + 'composites/' + EXP + '_' + nvar_ts + '-' + st + '_' + var + '-' + sf +'_std.nc')
        print(outfile)


DJF  DJF
/gws/nopw/j04/acsis/jmecking/CMIP6/composites/piControl_max_lats-DJF_zos-DJF_std.nc
DJF  MAM
/gws/nopw/j04/acsis/jmecking/CMIP6/composites/piControl_max_lats-MAM_zos-DJF_std.nc
DJF  JJA
/gws/nopw/j04/acsis/jmecking/CMIP6/composites/piControl_max_lats-JJA_zos-DJF_std.nc
DJF  SON
/gws/nopw/j04/acsis/jmecking/CMIP6/composites/piControl_max_lats-SON_zos-DJF_std.nc
MAM  DJF
/gws/nopw/j04/acsis/jmecking/CMIP6/composites/piControl_max_lats-DJF_zos-MAM_std.nc
MAM  MAM
/gws/nopw/j04/acsis/jmecking/CMIP6/composites/piControl_max_lats-MAM_zos-MAM_std.nc
MAM  JJA
/gws/nopw/j04/acsis/jmecking/CMIP6/composites/piControl_max_lats-JJA_zos-MAM_std.nc
MAM  SON
/gws/nopw/j04/acsis/jmecking/CMIP6/composites/piControl_max_lats-SON_zos-MAM_std.nc
JJA  DJF
/gws/nopw/j04/acsis/jmecking/CMIP6/composites/piControl_max_lats-DJF_zos-JJA_std.nc
JJA  MAM
/gws/nopw/j04/acsis/jmecking/CMIP6/composites/piControl_max_lats-MAM_zos-JJA_std.nc
JJA  JJA
/gws/nopw/j04/acsis/jmecking/CMIP6/composites/piControl_max_l

In [ ]:
    Models = list(Clist.keys())
    for mm in Models:
        Model = Clist[mm]
        print(Model.name)
        if ((Model.Ogrid != 'Unstructured')):                
            for var in vars:
                for EXP in EXPs:
                    ENSs = Model.getENSs(var=var,EXP=EXP,vtype=vtype,gtype='*')
                    for ENS in ENSs:
                        gtype = 'gn'
                        # Check if files needed for computation exist:
                        Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)
                        if ((len(Files) == 0) & (cmip == '6')):
                            gtype = 'gr'
                            Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)
                            if ((len(Files) == 0) & (cmip == '6')):
                                gtype = 'gr1'
                                Files = Model.getFiles(var=var,EXP=EXP,ENS=ENS,vtype=vtype,gtype=gtype)

                        if len(Files) != 0:
                            # Check if computation has already been completed:
                            outfile = ('/home/users/jmecking001/DATADIR/ACSIS/CMIP' + cmip + '/' + Model.name + '/' + EXP + '/' + ENS + '/' + var + '_' + Model.name + '_' + EXP + '_'
                                           + ENS + '_' + gtype + '_' + str(styr) + '01-' + str(fnyr) + '12_SC.nc')
                            
                            if not os.path.isfile(outfile):
                                Files.sort()
                                stf = int(Files[0].split('_')[-1].split('-')[0][:4])
                                fnf = int(Files[-1].split('_')[-1].split('-')[1][:4])
                                if ((stf <= styr) & (fnf >= fnyr)):
                                    print('computing: ' + outfile)
                                    # Generate submit script for lotus:
                                    lfile = ('slurm_files/SC_mean_' + var + '_' + Model.name + '_' +  EXP + '_' + ENS + '.slurm')

                                    line1 = '#!/bin/bash \n'
                                    line2 = '#SBATCH --account=short4hr \n'
                                    line3 = '#SBATCH -p short-serial-4hr \n'
                                    line4 = '#SBATCH -o %J.out \n'
                                    line5 = '#SBATCH -e %J.err \n'
                                    line6 = '#SBATCH --time 4:00:00 \n'
                                    line7 = '\n'
                                    line8 = 'conda activate working \n'
                                    line9 = ('python ../compute_means_SC.py ' + cmip + ' ' + Model.name + ' ' + EXP + ' ' + ENS + ' '
                                    + var + ' ' + vtype + ' ' + str(styr) + ' ' + str(fnyr) + ' ' + outfile + ' ' + gtype + ' ../' + lfile)

                                    # Out of memory models (second pass):
                                    #line2 = '#SBATCH -p high-mem \n'
                                    #line3 = '#SBATCH --mem=32000 \n'

                                    file = open(lfile,'w') 
                                    file.writelines([line1, line2, line3, line4, line5, line6, line7, line8, line9])

                                    file.close()